In [71]:
import pickle
import numpy as np
import os
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import csv
import json

def csv_to_geojson(filename, output_geojson, n_clusters=10):
    # for i in range(0,n_clusters):
    geojson = {
    "type": "FeatureCollection",
    "features": []
    }
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # print(int(row['cluster']))
            # if int(row['cluster'])==i:
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [float(row['lon']), float(row['lat'])]
                },
                "properties": {
                    "label": row['label'],
                    # "PC1": float(row['PC1']),
                    # "PC2": float(row['PC2']),
                    # "PC3": float(row['PC3']),
                    "cluster": int(row['cluster']),
                    # "path": row['path']
                }
            }
            geojson["features"].append(feature)
                # break
        geojson_file_path = f'{output_geojson}/cluster.geojson'
        # print(geojson_file_path)

        with open(geojson_file_path, 'w', encoding='utf-8') as geojsonfile:
            json.dump(geojson, geojsonfile, ensure_ascii=False, indent=4)

#### PCA (if needed)

In [43]:
def apply_pca(embeddings, n_components=20,start_component=5):
    pca = PCA(n_components)
    reduced_embeddings = pca.fit_transform(embeddings)
    print(f'Variance {pca.explained_variance_ratio_}')
    print(f'Variance {pca.explained_variance_ratio_[start_component:].sum()}')


    return reduced_embeddings

### Clustering algo

In [58]:
def apply_clustering(embeddings, labels, lat, lon, n_clusters, filename):
    print(embeddings.shape)
    print("K-Mean in process")
    kmeans = KMeans(n_clusters=n_clusters)
    clusters = kmeans.fit_predict(embeddings)

    df = pd.DataFrame({
    'label': labels,
    'cluster': clusters,
    'lat': lat,
    'lon':lon
    })

    df.to_csv(filename, index=False)

In [79]:
import ast

def convert_to_list(embedding_str):
    return ast.literal_eval(embedding_str)

filename=f'data/weather-embedding.csv'

data = pd.read_csv(filename)
data['embedding'] = data['embedding'].apply(convert_to_list)
embeddings = np.array(data['embedding'].tolist())
labels = data['cell-id'].values

# print("Labels shape:", labels.shape)
# print("Embeddings shape:", embeddings.shape)
# print("First embedding:", embeddings[0])


coord_data = pd.read_csv('data/location.csv')
coord_data_sub = coord_data[coord_data['cell-id'].isin(labels)]

lat = coord_data_sub['lat']
lon = coord_data_sub['lon']

SyntaxError: invalid syntax (<unknown>, line 1)

##### Apply PCA if needed

In [45]:
# n_components=16  # PCA components
# reduced_embeddings = apply_pca(embeddings, n_components=n_components)


In [69]:

output_geojson = '/home/savvas/SUPER-NAS/USERS/Chirag/TEMP-Folder/emb-cluster'
n_clusters = 7 

apply_clustering(embeddings, labels=labels, lat=lat, lon=lon, n_clusters=n_clusters, filename='weather-cluster.csv')


(4864, 32)
K-Mean in process


/home/savvas/anaconda3/envs/PyTorch/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [78]:
filename='weather-cluster.csv'
csv_to_geojson(filename,output_geojson, n_clusters=n_clusters)


In [77]:
os.path.exists('/home/savvas/SUPER-NAS/USERS/Chirag/TEMP-Folder/emb-cluster')

True